# Intact Medical Specialty Classification Model using NLP

### By: Daniyal, Hibah, Abhishek and Adam

### Step 1: Import libraries and read in the data

We'll add more libraries, as we move on.

In [31]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

df = pd.read_csv("new_train.csv", index_col=0)
print("Test size with duplicates: ", len(df))

Test size with duplicates:  3969


[nltk_data] Downloading package wordnet to /Users/adamyeo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/adamyeo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/adamyeo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adamyeo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Step 2: Pre-process our data

I think this is the most important step here, the ML model is only as good as its dataset, so we gotta make sure it's squeaky clean.

All of the basic pre-processing is done by the CountVectorizer, these tasks include:
- Tokenize (divide words individually)
- Remove stop-words (remove "the, and, to, or, ..."; other special characters)
- Lemmatize (convert similar words into its base root; eating, eats, ate => eat)

In [32]:
# Create labels/target values
y = df.labels
print("Label size: ", len(y))
y

Label size:  3969


0       0
1       1
2       1
3       2
4       0
       ..
3995    4
3996    1
3997    1
3998    5
3999    1
Name: labels, Length: 3969, dtype: int64

In [72]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df["transcription"], y, test_size=0.4, random_state=42)

# X_train: training data of features
print("X_train size: ", len(X_train))
# y_train: training data of label
print("y_train size: ", len(y_train))

# X_test: test data of features
print("X_test size: ", len(X_test))
# y_test: test data of label
print("y_test size: ", len(y_test))

# X_train
# y_train[:50]
# X_test
# y_test

X_train size:  2381
y_train size:  2381
X_test size:  1588
y_test size:  1588


In [86]:
# Instantiate the WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()


# Custom pre-processing function
def preprocess_data(text):
    text = text.lower()
    text = re.sub(r'\d|_', '', text) # removes digits and '_'
    text = wordnet_lemmatizer.lemmatize(text)
    return text

# , preprocessor=preprocess_data
# Initialize a CountVectorizer object
count_vectorizer = CountVectorizer(stop_words="english", preprocessor=preprocess_data, max_df=0.3, min_df=21, ngram_range=(1, 2))

print(type(count_vectorizer))

<class 'sklearn.feature_extraction.text.CountVectorizer'>


### Step 3: Fit and Transform the Data

Specifically, we must fit AND transform the feature training data and only transform the feature test data.
This is a preliminary step.

In fit_transform(), what happens is that we calculate the mean and variance of the training data and standardize the entire dataset (hence, transform). We only need transform() for the test data because we are using the mean and variance of the training data to standardize the test data.

In [90]:
# Fit and transform the TRAINING data using only the 'transciption' column values
count_train = count_vectorizer.fit_transform(X_train.values)
# Transform the TEST data using only the 'transciption' column values
count_test = count_vectorizer.transform(X_test.values)


# Print number of words processing
print("Number of words: ", len(count_vectorizer.get_feature_names_out())) # number of test data from split
# Print the features (individual tokens) of the count_vectorizer
print(count_vectorizer.get_feature_names_out()[:500])

Number of words:  4916
['abc' 'abcd' 'abcd general' 'abcd hospital' 'abdomen'
 'abdomen insufflated' 'abdomen pelvis' 'abdomen prepped' 'abdomen soft'
 'abdominal' 'abdominal aorta' 'abdominal cavity' 'abdominal pain'
 'abdominal wall' 'abds' 'ability' 'able' 'able walk' 'abnormal'
 'abnormalities' 'abnormalities noted' 'abnormality' 'abscess' 'absence'
 'absent' 'abuse' 'ac' 'access' 'accessory' 'accident' 'accommodate'
 'accommodation' 'accompanied' 'accomplished' 'according'
 'according patient' 'ace' 'achieve' 'achieved' 'acid' 'acquired' 'active'
 'active bleeding' 'activities' 'activities daily' 'activity' 'actually'
 'acuity' 'acute' 'acute distress' 'adaptic' 'add' 'added' 'addition'
 'additional' 'additionally' 'address' 'addressed' 'adenocarcinoma'
 'adenoidectomy' 'adenopathy' 'adenopathy thyromegaly' 'adequate'
 'adequate anesthesia' 'adequate general' 'adequate sedation' 'adequately'
 'adherent' 'adhesions' 'adjacent' 'adjusted' 'administered'
 'administered patient' 'admi

### Step 4: Train our models here

We used the Multinomial Naive Bayes to classify our labels

In [88]:
# Instantiate a Multinomial Naive Bayes classifier
nb_clf = MultinomialNB(alpha=0.4)
# Fit the classifier to the training data
nb_clf.fit(count_train, y_train)
# Create the predicted tags
pred = nb_clf.predict(count_test)

# Print the predictions for each row of the dataset (1001 rows)
print("Number of predictions: ", len(pred)) # Equal to the number of test data (when it got split)
print(pred)

Number of predictions:  1588
[ 7  1 34 ...  1  1  1]


### Step 4: Evaluate the model

We will create an accuracy score and also a confusion matrix.

Precision = TP/(TP + FP)

Recall = TP/(TP+FN)

F1 Score = 2*(Recall * Precision) / (Recall + Precision)

In [89]:
# Calculate the accuracy score
score = metrics.accuracy_score(y_test, pred)
# Calculate the confusion matrix
# conf_matrix = metrics.confusion_matrix(y_test, pred)

print(score)
print(classification_report(y_test, pred))

0.3494962216624685
              precision    recall  f1-score   support

           0       0.26      0.15      0.19        33
           1       0.34      0.18      0.23       310
           2       0.31      0.34      0.32        77
           3       0.34      0.59      0.43        17
           4       0.34      0.42      0.38        60
           5       0.38      0.46      0.41        74
           6       0.46      0.34      0.39       115
           7       0.52      0.37      0.43       128
           8       0.23      0.10      0.14        29
           9       0.47      0.57      0.52        42
          10       0.21      0.19      0.20        97
          11       0.13      0.11      0.12        28
          12       0.14      0.29      0.19         7
          13       0.17      0.25      0.21        44
          14       0.33      0.40      0.36         5
          15       0.53      0.74      0.62        23
          16       0.30      0.35      0.32       169
        